# Gate Fidelity Benchmarking

In [ ]:
import matplotlib.pyplot as plt
from equal1.benchmarking.device_characterization.gate_fidelity import Gate1qFidelity
import pandas as pd
import seaborn as sns
from rich import progress

In [ ]:
hardware = {
    "hpc-conservative",
    "hpc-gamma1",
    "hpc-gamma2",
}
num_qubits = 16

## Single-Qubit Gates Fidelity

In [ ]:
gates_to_benchmark = ["x", "sx"]

experiments = [
    Gate1qFidelity(device_name=hw, qubit_idx=q, gate_to_benchmark=g)
    for hw in hardware
    for q in range(num_qubits)
    for g in gates_to_benchmark
]

In [ ]:
results = []

for experiment in progress.track(
    experiments, description="Running Experiments", auto_refresh=False
):
    experiment.run(
        simulate=True,
        runtime_options={
            "backend": "TensorNet_MPS",
            "simulation_platform": "GPU",
        },
    )

    fidelity = experiment.analyse_results()

    results.append(
        {
            "device": experiment.device_name,
            "qubit_idx": experiment.qubit_idx,
            "gate_name": experiment.gate_to_benchmark.name,
            "fidelity": fidelity,
        }
    )

In [ ]:
df = pd.DataFrame(results)
df.to_csv("single_qubit_gate_fidelity_results.csv", index=False)

In [ ]:
for device in df["device"].unique():
    fig, ax = plt.subplots(figsize=(24, 6))
    sns.barplot(
        data=df[df["device"] == device],
        x="qubit_idx",
        hue="gate_name",
        y="fidelity",
        ax=ax,
    )
    ax.set_ylabel("Gate Fidelity")
    ax.set_xlabel("Qubit Index")
    for container in ax.containers:
        ax.bar_label(container, fmt="%.5f")
    ax.set_title(f"Single Qubit Gate Fidelity on {device}")
    plt.show()